# Train Pixel Level Annotation Model

### This notebook uses pixel level annotations to train a Random Forest Classifier to predict labels

We assume Pixel level annotations are available, as produced by the "../data/annotations/transform_polygon_annotations_to_pixels.ipynb" notebook. Feature selection and grid_search for optimal parameters has been done in a separate notebook ("Coepelduynen/make_train_model_on_annotations_coepelduynen.ipynb") and those outcomes are taken as given in this notebook.

Change the set Variables cell below as desired and then run the entire notebook to get cross_validation results as well as a final model trained on all data.

Date: 2024-01-12\
Author: Pieter Kouyzer

In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
%matplotlib notebook
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pprint

from training.train import train_imbalanced_model, cross_validation_balance_on_date
from training.utils import get_cross_validation_results_filepath, get_model_filepath
from training.metric_calculation import calculate_average_metrics, get_metrics

2024/02/06 10:08:10 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


In [5]:
# Set Variables
location = "Coepelduynen"
satellite_constellation = "PNEO"

if location == "Voornes Duin":
    annotated_pixels_filepath = "C:/Users/pzhadmin/Data/remote-sensing/annotations/Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet"
elif location == "Coepelduynen":
    annotated_pixels_filepath = "C:/repos/satellite-images-nso-datascience/data/annotations/annotations_pixel_dataframes/annotaties_coepelduynen_to_pixel_2023.parquet"

In [6]:
# Optimal parameters and features Voornes
if location == "Voornes Duin":
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd', 'ndvi','re_ndvi', 'height']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "max_features": "auto", 
        "bootstrap": False
    }

elif location == "Coepelduynen":
    # Optimal parameters and features
    selected_features = ['r','g','b','n','e','d','ndvi','re_ndvi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

In [7]:
df = pd.read_parquet(annotated_pixels_filepath)
df

,r,g,b,n,e,d,ndvi,re_ndvi,label,image,date,season,annotation_no
0,832.0,766.0,690.0,1502.0,1138.0,760.0,128.0,115.0,Sand,20230513_104139_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230513_104139,Spring,31_20230513_annotations
1,875.0,792.0,708.0,1546.0,1181.0,768.0,127.0,114.0,Sand,20230513_104139_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230513_104139,Spring,31_20230513_annotations
2,852.0,778.0,698.0,1454.0,1131.0,761.0,126.0,114.0,Sand,20230513_104139_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230513_104139,Spring,31_20230513_annotations
3,796.0,738.0,668.0,1285.0,1033.0,738.0,123.0,112.0,Sand,20230513_104139_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230513_104139,Spring,31_20230513_annotations
4,780.0,730.0,662.0,1260.0,1014.0,735.0,123.0,113.0,Sand,20230513_104139_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230513_104139,Spring,31_20230513_annotations
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156367,471.0,517.0,558.0,862.0,658.0,691.0,129.0,116.0,Vegetation,20230910_105008_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230910_105008,Fall,30_Annotations_Coepelduynen_2023
156368,467.0,514.0,557.0,844.0,648.0,691.0,128.0,116.0,Vegetation,20230910_105008_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230910_105008,Fall,30_Annotations_Coepelduynen_2023
156369,420.0,477.0,532.0,700.0,562.0,673.0,125.0,114.0,Vegetation,20230910_105008_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230910_105008,Fall,30_Annotations_Coepelduynen_2023
156370,430.0,485.0,537.0,736.0,582.0,677.0,126.0,115.0,Vegetation,20230910_105008_PNEO-03_1_1_30cm_RD_12bit_RGBN...,20230910_105008,Fall,30_Annotations_Coepelduynen_2023


In [8]:
# This is to give an indication of the amount of data points per image
df['date'].value_counts()

date
20230513_104139    392936
20230402_105321    156372
20230601_105710    156372
20230908_110020    156372
20230910_105008    156372
Name: count, dtype: int64

In [9]:
# This is to give an indication of the amount of data points per label
df['label'].value_counts()

label
Vegetation    898609
Sand          119415
Asphalt          400
Name: count, dtype: int64

In [10]:
# Make sure there is no Ashpalt.
df = df[df['label'] != "Asphalt"]

### Cross Validation

We do cross-validation, where the folds are decided by the 'date' column. This is to avoid pixels from the same image from ending up in both the train and test datasets. We display the metrics averaged over the folds and write the results to a pickle.

In [12]:
model = RandomForestClassifier(**optimal_parameters)
scaler = StandardScaler()

In [14]:
if location == "Voornes Duin":
    cv =5 
    random_state = 1337
    sampling_type_boundary = 100000
elif location == "Coepelduynen":
    cv =4 
    random_state = 1337
    sampling_type_boundary = 898609

In [15]:
results = cross_validation_balance_on_date(data=df, model=model, cv=cv, features=selected_features, random_state=random_state, sampling_type_boundary=sampling_type_boundary , scaler=scaler)

---------fold: 1
Picked hold out dates: 
['20230402_105321']
Oversampling to rebalance dataset


2024/02/06 10:24:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3bef4694515745ad91d29be953b60917', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/02/06 10:24:46 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/02/06 10:24:46 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2024/02/06 10:24:46 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Fitting model


2024/02/06 10:24:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


<IPython.core.display.Javascript object>

...

2024/02/06 10:33:02 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Calculating train metrics
Calculating test metrics
{'Sand': {'precision': 0.9445669034206874, 'recall': 1.0, 'f1-score': 0.9714933456484319, 'support': 11570}, 'Vegetation': {'precision': 1.0, 'recall': 0.9953082461546966, 'f1-score': 0.9976486069987706, 'support': 144722}}


In [16]:
calculate_average_metrics(results=results)

,precision,recall,f1-score
Sand,0.990585,0.945913,0.967059
Vegetation,0.992399,0.998599,0.995464


In [ ]:
cross_validation_results_filepath = get_cross_validation_results_filepath(location=location, satellite_constellation=satellite_constellation, df=df)
print(f"Saving to {cross_validation_results_filepath}")
with open(cross_validation_results_filepath, "wb") as file:
    pickle.dump(results, file)

### Export Definitive model.

Trains a Random Forest Classifier model on all data and writes it to a pickle file for later use. This is the definitive model output by this notebook.

In [17]:
final_model = RandomForestClassifier(**optimal_parameters)
final_scaler = StandardScaler()

train_imbalanced_model(
    X_train=df[selected_features], 
    y_train=df["label"], 
    model=final_model, 
    random_state=1337, 
    sampling_type_boundary=sampling_type_boundary ,
    scaler=final_scaler
)
pprint.pprint(get_metrics(y=df["label"], X=df[selected_features], model=final_model, scaler=final_scaler))

Oversampling to rebalance dataset


2024/02/06 10:36:18 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4279fd7bbf414b09aff7783f6d8bc68f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2024/02/06 10:36:18 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/02/06 10:36:18 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2024/02/06 10:36:18 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Fitting model


2024/02/06 10:36:27 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

{'Sand': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 119415},
 'Vegetation': {'f1-score': 1.0,
                'precision': 1.0,
                'recall': 1.0,
                'support': 898609}}


In [18]:
final_artefact = {
    "model": final_model,
    "scaler": final_scaler
}

In [19]:
final_model_filepath = get_model_filepath(location=location, satellite_constellation=satellite_constellation, df=df)
print(f"Saving to {final_model_filepath}")
with open(final_model_filepath, "wb") as file:
    pickle.dump(final_artefact, file)

Saving to ../saved_models/PNEO_Coepelduynen_20230402_105321_to_20230910_105008_random_forest_classifier.sav
